In [1]:
casino = []
with open('kasyno10K.txt') as f:
    casino = [int(i) for i in f.read() if i != '\n' ]

In [2]:
def forward_backward(p_transition, emision, data):
    alpha = [[0 for i in range(10000)],
            [0 for i in range(10000)]] # p-stwo, ze w i-tym kroku bedziemy w (0,1) stanie
    alpha[0][0] = 1 
    alpha[0][1] = 0 # bo startujemy z kostki 'poprawnej'
    def count_alpha(T):
        for t in range(1,T):
            for j in range(2):
                a0 = alpha[0][t-1] * p_transition[0][j] * emision[j][int(data[t] - 1)]
                a1 = alpha[1][t-1] * p_transition[1][j] * emision[j][int(data[t] - 1)]
                alpha[j][t] = a0 + a1  
                #alpha[j][t] *= 6
            n = alpha[0][t] + alpha[1][t]
            alpha[0][t] = alpha[0][t] / n
            alpha[1][t] = alpha[1][t] / n

    beta = [[0 for i in range(10000)],
            [0 for i in range(10000)]] # p-stwo, ze w i-tym kroku bedziemy w (0,1) stanie
    beta[0][-1] = 1
    beta[1][-1] = 1
    def count_beta(T):
        for t in range(T-2,-1,-1):
            for j in range(2):
                b0 = beta[0][t+1] * p_transition[j][0] * emision[0][int(data[t+1] - 1)]
                b1 = beta[1][t+1] * p_transition[j][1] * emision[1][int(data[t+1] - 1)]
                beta[j][t] = b0 + b1  
                #beta[j][t] *= 6
            n = beta[0][t] + beta[1][t]
            beta[0][t] = beta[0][t] / n
            beta[1][t] = beta[1][t] / n
    count_alpha(10000)
    count_beta(10000)
    ab = [[0 for i in range(10000)],
        [0 for i in range(10000)]]
    for i in range(len(ab[0])):
        n = alpha[0][i] * beta[0][i] + alpha[1][i] * beta[1][i]
        ab[0][i] = alpha[0][i] * beta[0][i] / n
        ab[1][i] = alpha[1][i] * beta[1][i] / n
    return alpha, beta, ab

In [7]:
import random
from tqdm.notebook import tqdm
def r_d():
    r = [random.randint(1,6) for i in range(6)]
    #p = random.choices([i for i in range(1,10)], k=6)
    summ = sum(r)
    return [i/summ for i in r]
def Baum_Welch(data, p0, p1):
    p_transition = [[1 - p0, p0], [p1, 1 - p1]]
    emision = [r_d(),r_d()]
    #print(emision)
    it = 300
    for _ in tqdm(range(it)):
        ksi = [[[0,0],[0,0]] for i in range(10000 - 1)]
        alpha, beta, gamma = forward_backward(p_transition, emision, data)
        for t in range(10000 - 1):
            summ = 0
            for i in range(2):
                for j in range(2):
                    summ += alpha[i][t] * p_transition[i][j] * beta[j][t + 1] * emision[j][data[t + 1] - 1]
                #print(summ, alpha[i][t], beta[i][t])
            for i in range(2):
                for j in range(2):
                    ksi[t][i][j] = (alpha[i][t] * p_transition[i][j] * emision[j][data[t + 1] - 1] * beta[j][t + 1])/ summ
        
        new_transition = [[0,0],[0,0]]
        for i in range(2):
            for j in range(2):
                sum1 = 0
                sum2 = 0
                for t in range(10000 - 1):
                    sum1 += ksi[t][i][j]
                    sum2 += gamma[i][t]
                new_transition[i][j] = sum1 / sum2
        new_emision = [[0 for i in range(6)] for j in range(2)]
        for i in range(2):
            for k in range(6):
                sum1 = 0
                sum2 = 0
                for t in range(10000):
                    if data[t] == k + 1:
                        sum1 += gamma[i][t]
                    sum2 += gamma[i][t]
                new_emision[i][k] = sum1 / sum2
        p_transition = new_transition
        emision = new_emision
        
        for i in range(2):
            p = sum(p_transition[i])
            for j in range(2):
                p_transition[i][j] /= p
        #print(p_transition)
        #print(emision)
    
    return p_transition, emision
    

In [8]:
Baum_Welch(casino, 0.05 , 0.05)

([[0.950504289736249, 0.04949571026375104],
  [0.03428011104107674, 0.9657198889589234]],
 [[0.2116636516896978,
   0.19726275852304212,
   0.29808526441389377,
   0.08817544334849349,
   0.10562805252132003,
   0.0991848295035517],
  [0.0894988099976941,
   0.09659908212299737,
   0.10376688767770015,
   0.09871290868081946,
   0.2059687075597558,
   0.4054536039610376]])

# RESULT

dobra kostka: p_zmiany ~ 0.03 ,
[0.20, 0.20, 0.30, 0.10, 0.10, 0.10] <br>
zla kostka: p_zmiany ~ 0.05 ,
[0.10, 0.10, 0.10, 0.10, 0.20, 0.40]

In [9]:
Baum_Welch(casino, 0.1, 0.1)

([[0.96537510776007, 0.03462489223993013],
  [0.049753141654517774, 0.9502468583454822]],
 [[0.08959262483257437,
   0.09649711802965275,
   0.10385479972775376,
   0.09875674072200365,
   0.20589256170586528,
   0.40540615498214977],
  [0.2115341181952381,
   0.19741476979454242,
   0.29796772049736153,
   0.08811167136997353,
   0.10573313287585709,
   0.0992385872670315]])

dobra kostka: p_zmiany ~ 0.03 ,
[0.09, 0.10, 0.10, 0.10, 0.20, 0.41] <br>
zla kostka: p_zmiany ~ 0.05 ,
[0.20, 0.20, 0.30, 0.10, 0.10, 0.10]

In [10]:
Baum_Welch(casino, 0.15, 0.15)

([[0.96537510776007, 0.03462489223993014],
  [0.049753141654517774, 0.9502468583454822]],
 [[0.08959262483257437,
   0.09649711802965272,
   0.1038547997277537,
   0.09875674072200366,
   0.20589256170586534,
   0.4054061549821499],
  [0.21153411819523804,
   0.1974147697945424,
   0.2979677204973616,
   0.08811167136997354,
   0.10573313287585707,
   0.0992385872670315]])

dobra kostka: p_zmiany ~ 0.03 ,
[0.09, 0.10, 0.10, 0.10, 0.20, 0.41] <br>
zla kostka: p_zmiany ~ 0.05 ,
[0.20, 0.20, 0.30, 0.10, 0.10, 0.10]

In [11]:
Baum_Welch(casino, 0.30, 0.30)

([[0.9653751077600699, 0.0346248922399301],
  [0.04975314165451777, 0.9502468583454823]],
 [[0.08959262483257435,
   0.09649711802965275,
   0.10385479972775376,
   0.09875674072200362,
   0.20589256170586526,
   0.40540615498214966],
  [0.21153411819523812,
   0.19741476979454248,
   0.29796772049736164,
   0.08811167136997357,
   0.1057331328758571,
   0.0992385872670315]])

dobra kostka: p_zmiany ~ 0.03 ,
[0.09, 0.10, 0.10, 0.10, 0.20, 0.41] <br>
zla kostka: p_zmiany ~ 0.05 ,
[0.20, 0.20, 0.30, 0.10, 0.10, 0.10]

In [12]:
Baum_Welch(casino, 0.40, 0.40)

([[0.9653751077600699, 0.034624892239930115],
  [0.049753141654517795, 0.9502468583454821]],
 [[0.08959262483257437,
   0.09649711802965273,
   0.10385479972775373,
   0.09875674072200365,
   0.20589256170586523,
   0.4054061549821497],
  [0.2115341181952381,
   0.19741476979454245,
   0.2979677204973617,
   0.08811167136997355,
   0.10573313287585705,
   0.09923858726703148]])

dobra kostka: p_zmiany ~ 0.03 ,
[0.09, 0.10, 0.10, 0.10, 0.20, 0.41] <br>
zla kostka: p_zmiany ~ 0.05 ,
[0.20, 0.20, 0.30, 0.10, 0.10, 0.10]

In [15]:
Baum_Welch(casino, 0.70, 0.40)

([[0.9653751077600699, 0.03462489223993011],
  [0.04975314165451778, 0.9502468583454822]],
 [[0.08959262483257434,
   0.09649711802965272,
   0.10385479972775373,
   0.09875674072200362,
   0.20589256170586526,
   0.40540615498214966],
  [0.21153411819523818,
   0.19741476979454248,
   0.29796772049736164,
   0.08811167136997354,
   0.1057331328758571,
   0.09923858726703151]])